In [1]:
import pandas as pd

In [10]:
datas = pd.read_csv('./DATA/videos/녹두로_포켓몬스바_20230402.csv')

# calculate the date difference between rows
datas['date_diff'] = pd.to_datetime(datas['date'], format='%Y-%m-%dT%H:%M:%SZ').diff().apply(lambda x: x.total_seconds() / 3600)

# set the first date_diff value to 0
datas.loc[datas.index[0], 'date_diff'] = 0

datas['view_per_first'] = datas['views'] / datas.loc[datas.index[0], 'views']
datas['likes_per_first'] = datas['likes'] / datas.loc[datas.index[0], 'likes']
datas['likes_per_view'] = datas['likes'] / datas.loc[datas.index[0], 'views']


datas.head()

,Unnamed: 0,title,category_id,views,likes,favorites,comments,date,tags,date_diff,view_per_first,likes_per_first,likes_per_view
0,0,포켓몬스터 스칼렛 바이올렛 #1,20,192132,1423,0,Nan,2022-11-17T22:47:06Z,"['녹두로', '포켓몬스터', 'pokemon', '포켓몬', '닌텐도 스위치', ...",0.000000,1.000000,1.000000,0.007406
1,1,포켓몬스터 스칼렛 바이올렛 #2,20,110304,797,0,Nan,2022-11-19T00:36:46Z,"['녹두로', '포켓몬스터', 'pokemon', '포켓몬', '닌텐도 스위치', ...",25.827778,0.574105,0.560084,0.004148
2,2,포켓몬스터 스칼렛 바이올렛 #3,20,111456,689,0,Nan,2022-11-20T00:13:53Z,"['녹두로', '포켓몬스터', 'pokemon', '포켓몬', '닌텐도 스위치', ...",23.618611,0.580101,0.484188,0.003586
3,3,포켓몬스터 스칼렛 바이올렛 #4,20,83007,532,0,Nan,2022-11-24T09:48:43Z,"['녹두로', '포켓몬스터', 'pokemon', '포켓몬', '닌텐도 스위치', ...",105.580556,0.432031,0.373858,0.002769
4,4,포켓몬스터 스칼렛 바이올렛 #5,20,54895,371,0,Nan,2022-11-25T23:09:57Z,"['녹두로', '포켓몬스터', 'pokemon', '포켓몬', '닌텐도 스위치', ...",37.353889,0.285715,0.260717,0.001931
